In [1]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# 
train=pd.read_csv('../data/train.csv')
test=pd.read_csv('../data/test.csv')
#more about kinetic features  developed  by Daia Alexandru    here  on the next  blog  please  read  last article :
#https://alexandrudaia.quora.com/

##############################################creatinng   kinetic features for  train #####################################################
def  kinetic(row):
    probs=np.unique(row,return_counts=True)[1]/len(row)
    kinetic=np.sum(probs**2)
    return kinetic
    

first_kin_names=[col for  col in train.columns  if '_ind_' in col]
subset_ind=train[first_kin_names]
kinetic_1=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_1.append(k)
second_kin_names= [col for  col in train.columns  if '_car_' in col and col.endswith('cat')]
subset_ind=train[second_kin_names]
kinetic_2=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_2.append(k)
third_kin_names= [col for  col in train.columns  if '_calc_' in col and  not col.endswith('bin')]
subset_ind=train[second_kin_names]
kinetic_3=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_3.append(k)
fd_kin_names= [col for  col in train.columns  if '_calc_' in col and  col.endswith('bin')]
subset_ind=train[fd_kin_names]
kinetic_4=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_4.append(k)
train['kinetic_1']=np.array(kinetic_1)
train['kinetic_2']=np.array(kinetic_2)
train['kinetic_3']=np.array(kinetic_3)
train['kinetic_4']=np.array(kinetic_4)

############################################reatinng   kinetic features for  test###############################################################

first_kin_names=[col for  col in test.columns  if '_ind_' in col]
subset_ind=test[first_kin_names]
kinetic_1=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_1.append(k)
second_kin_names= [col for  col in test.columns  if '_car_' in col and col.endswith('cat')]
subset_ind=test[second_kin_names]
kinetic_2=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_2.append(k)
third_kin_names= [col for  col in test.columns  if '_calc_' in col and  not col.endswith('bin')]
subset_ind=test[second_kin_names]
kinetic_3=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_3.append(k)
fd_kin_names= [col for  col in test.columns  if '_calc_' in col and  col.endswith('bin')]
subset_ind=test[fd_kin_names]
kinetic_4=[]
for row in range(subset_ind.shape[0]):
    row=subset_ind.iloc[row]
    k=kinetic(row)
    kinetic_4.append(k)
test['kinetic_1']=np.array(kinetic_1)
test['kinetic_2']=np.array(kinetic_2)
test['kinetic_3']=np.array(kinetic_3)
test['kinetic_4']=np.array(kinetic_4)

##################################################################end  of kinetics ############################################################################
from sklearn import *
import xgboost as xgb
from multiprocessing import *
import numpy as np
import pandas as pd
from sklearn import *
import xgboost as xgb
import lightgbm as lgb
from multiprocessing import *


d_median = train.median(axis=0)
d_mean = train.mean(axis=0)

def transform_df(df):
    df = pd.DataFrame(df)
    dcol = [c for c in df.columns if c not in ['id','target']]
    df['ps_car_13_x_ps_reg_03'] = df['ps_car_13'] * df['ps_reg_03']
    df['negative_one_vals'] = np.sum((df[dcol]==-1).values, axis=1)
    for c in dcol:
        if '_bin' not in c:
            df[c+str('_median_range')] = (df[c].values > d_median[c]).astype(int)
            df[c+str('_mean_range')] = (df[c].values > d_mean[c]).astype(int)
    return df

def multi_transform(df):
    print('Init Shape: ', df.shape)
    p = Pool(cpu_count())
    df = p.map(transform_df, np.array_split(df, cpu_count()))
    df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
    p.close(); p.join()
    print('After Shape: ', df.shape)
    return df

def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_xgb(pred, y):
    #y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}
x1, x2, y1, y2 = model_selection.train_test_split(train, train['target'], test_size=0.25, random_state=99)

#keep dist
x1 = multi_transform(x1)
y1 = x1['target']
x2 = multi_transform(x2)
y2 = x2['target']
test = multi_transform(test)

col = [c for c in x1.columns if c not in ['id','target']]
x1 = x1[col]
x2 = x2[col]

watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
model = xgb.train(params, xgb.DMatrix(x1, y1), 5000,  watchlist, verbose_eval=50, early_stopping_rounds=100)
print(gini_xgb(model.predict(xgb.DMatrix(x2), ntree_limit=model.best_ntree_limit), y2))
test['target'] = model.predict(xgb.DMatrix(test[col]), ntree_limit=model.best_ntree_limit)
test[['id','target']].to_csv('uberKinetics.csv', index=False, float_format='%.5f')

print 'David'
print model.feature_importances_

/Users/davidsutton/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/davidsutton/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/davidsutton/anaconda/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be 

('Init Shape: ', (446409, 63))
('After Shape: ', (446409, 153))
('Init Shape: ', (148803, 63))
('After Shape: ', (148803, 153))
('Init Shape: ', (892816, 62))
('After Shape: ', (892816, 152))
[0]	train-auc:0.593729	valid-auc:0.59144
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 100 rounds.
[50]	train-auc:0.622302	valid-auc:0.619788
[100]	train-auc:0.625791	valid-auc:0.621671
[150]	train-auc:0.631683	valid-auc:0.624888
[200]	train-auc:0.638737	valid-auc:0.628223
[250]	train-auc:0.644499	valid-auc:0.630981
[300]	train-auc:0.649426	valid-auc:0.633193
[350]	train-auc:0.653725	valid-auc:0.635089
[400]	train-auc:0.65736	valid-auc:0.636618
[450]	train-auc:0.660508	valid-auc:0.637507
[500]	train-auc:0.66334	valid-auc:0.638086
[550]	train-auc:0.666081	valid-auc:0.638287
[600]	train-auc:0.668572	valid-auc:0.638422
[650]	train-auc:0.671136	valid-auc:0.638938
[700]	train-auc:0.67348	valid-auc:0.639113
[750]	train

AttributeError: 'Booster' object has no attribute 'feature_importances_'

In [3]:
xgb_params= {'learning_rate': 0.07,
             'n_estimators':1000, 
             'max_depth': 4, 
             'nthread':8
            }
xgbm1= xgb.sklearn.XGBClassifier(**xgb_params)
xgbm1.fit(x1.as_matrix(),y1,eval_metric='auc',early_stopping_rounds=50,eval_set=[(x2.as_matrix(),y2)])
y_pred= xgbm1.predict_proba(x2.as_matrix())[:,1]

[0]	validation_0-auc:0.592618
Will train until validation_0-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.59697
[2]	validation_0-auc:0.608103
[3]	validation_0-auc:0.610974
[4]	validation_0-auc:0.610521
[5]	validation_0-auc:0.612097
[6]	validation_0-auc:0.612656
[7]	validation_0-auc:0.612521
[8]	validation_0-auc:0.614467
[9]	validation_0-auc:0.615201
[10]	validation_0-auc:0.615846
[11]	validation_0-auc:0.616502
[12]	validation_0-auc:0.616824
[13]	validation_0-auc:0.617111
[14]	validation_0-auc:0.617189
[15]	validation_0-auc:0.61764
[16]	validation_0-auc:0.618033
[17]	validation_0-auc:0.618152
[18]	validation_0-auc:0.619087
[19]	validation_0-auc:0.619106
[20]	validation_0-auc:0.619012
[21]	validation_0-auc:0.618855
[22]	validation_0-auc:0.619248
[23]	validation_0-auc:0.619486
[24]	validation_0-auc:0.620066
[25]	validation_0-auc:0.619934
[26]	validation_0-auc:0.620184
[27]	validation_0-auc:0.620407
[28]	validation_0-auc:0.620416
[29]	validation_0-auc:0.62077
[30]	validation_0-a

NameError: name 'sklearn' is not defined

In [4]:
import sklearn.metrics
print 'Gini score= ',2.*sklearn.metrics.roc_auc_score(y2,y_pred)-1.

print 'David'
print xgbm1.feature_importances_

 Gini score=  0.27728467349
David
[ 0.03651939  0.01532913  0.06559964  0.01375113  0.04215509  0.0087917
  0.00946799  0.00405771  0.00473399  0.00022543  0.          0.00180343
  0.          0.001578    0.03516682  0.00631199  0.02412083  0.001578
  0.02772768  0.02795311  0.04643823  0.02276826  0.00247971  0.01217313
  0.00563571  0.00450857  0.01983769  0.01623084  0.00428314  0.01848512
  0.00045086  0.02569883  0.01172227  0.02321912  0.09535617  0.03358882
  0.01803426  0.0135257   0.01577998  0.01330027  0.00969342  0.01172227
  0.01082056  0.01735798  0.0151037   0.01262399  0.02457169  0.01330027
  0.01149684  0.01217313  0.02119026  0.00450857  0.00270514  0.00247971
  0.00360685  0.00518485  0.00090171  0.          0.          0.
  0.00180343  0.07822362  0.01014427  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.

In [15]:
info= pd.DataFrame({'feature':x1.columns.values,'importance':xgbm1.feature_importances_},
                   columns=['feature','importance'])

In [17]:
info[ map(lambda x: 'kinetic' in x, info.feature.values)]

,feature,importance
57,kinetic_1,0.000000
58,kinetic_2,0.000000
59,kinetic_3,0.000000
60,kinetic_4,0.001803
143,kinetic_1_median_range,0.000000
144,kinetic_1_mean_range,0.000000
145,kinetic_2_median_range,0.000000
146,kinetic_2_mean_range,0.000000
147,kinetic_3_median_range,0.000000
148,kinetic_3_mean_range,0.000000


In [18]:
info.importance

0      0.036519
1      0.015329
2      0.065600
3      0.013751
4      0.042155
5      0.008792
6      0.009468
7      0.004058
8      0.004734
9      0.000225
10     0.000000
11     0.001803
12     0.000000
13     0.001578
14     0.035167
15     0.006312
16     0.024121
17     0.001578
18     0.027728
19     0.027953
20     0.046438
21     0.022768
22     0.002480
23     0.012173
24     0.005636
25     0.004509
26     0.019838
27     0.016231
28     0.004283
29     0.018485
         ...   
121    0.000000
122    0.000000
123    0.000000
124    0.000000
125    0.000000
126    0.000000
127    0.000000
128    0.000000
129    0.000000
130    0.000000
131    0.000000
132    0.000000
133    0.000000
134    0.000000
135    0.000000
136    0.000000
137    0.000000
138    0.000000
139    0.000000
140    0.000000
141    0.000000
142    0.000000
143    0.000000
144    0.000000
145    0.000000
146    0.000000
147    0.000000
148    0.000000
149    0.000000
150    0.000000
Name: importance, dtype: